In [1]:
# import libraries
# ================

# for date and time opeations
from datetime import datetime
# for file and folder operations
import os
# for regular expression opeations
import re
# for listing files in a folder
import glob
# for getting web contents
import requests 
# storing and analysing data
import pandas as pd
# for scraping web contents
from bs4 import BeautifulSoup

import re

import numpy as np

In [2]:
# get data
# ========

# link at which web data recides
link = 'https://www.worldometers.info/coronavirus/'
# get web data
req = requests.get(link)
# parse web data
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# find the table
# ==============
# our target table is the last table in the page

# get the table head
# table head may contain the column names, titles, subtitles
thead = soup.find_all('thead')[-1]
# print(thead)

# get all the rows in table head
# it usually have only one row, which has the column names
head = thead.find_all('tr')
# print(head)

# get the table tbody
# it contains the contents
tbody = soup.find_all('tbody')[0]
# print(tbody)

# get all the rows in table body
# each row is each state's entry
body = tbody.find_all('tr')
# print(body)

In [4]:
# get the table contents
# ======================

# container for header rows / column title
head_rows = []
# container for table body / contents
body_rows = []

# loop through the head and append each row to head
for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

# loop through the body and append each row to body
for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
# print(head_rows)

In [5]:
# save contents in a dataframe
# ============================
    
# skip last 3 rows, it contains unwanted info
# head_rows contains column title
df_bs = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         

# Drop 'S. No.' column
# df_bs.drop('S. No.', axis=1, inplace=True)

# there are 36 states+UT in India
df_bs.head(10)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,,\nNorth America\n,"2,307,214","+3,878","136,769",+384,"946,457","+3,180","1,223,988","19,439",,,,,,North America,\n,,
1,,\nSouth America\n,"1,176,584",+308,"51,337",+10,"560,034",+73,"565,213","11,792",,,,,,South America,\n,,
2,,\nEurope\n,"2,109,603",+425,"179,384",+24,"1,089,772",+857,"840,447","7,116",,,,,,Europe,\n,,
3,,\nAsia\n,"1,399,137","+6,655","35,763",+124,"844,653","+1,739","518,721","14,957",,,,,,Asia,\n,,
4,,\nAfrica\n,"199,000",,"5,390",,"88,212",,"105,398",490,,,,,,Africa,\n,,
5,,\nOceania\n,"8,877",+2,124,,"8,307",+13,446,2,,,,,,Australia/Oceania,\n,,
6,,\n\n,721,,15,,651,,55,4,,,,,,,\n,,
7,,World,"7,201,136","+11,268","408,782",+542,"3,538,086","+5,862","3,254,268","53,800",924,52.4,,,,All,\n,,
8,1,USA,"2,026,597",+104,"113,061",+6,"773,505",+25,"1,140,031","16,907","6,125",342,"21,727,338","65,664","330,885,824",North America,163,"2,927",15
9,2,Brazil,"710,887",,"37,312",,"325,602",,"347,973","8,318","3,346",176,"999,836","4,706","212,467,494",South America,299,"5,694",213


In [6]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Congo, DRC, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bermuda, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, USA, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Vatican City, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, UK, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, UAE, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Hong Kong, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, S. Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'

In [7]:
df_bs.tail()

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
212,205,Seychelles,11,,,,11,,0,,112,,,,"98,310",Africa,"8,937",,
213,206,MS Zaandam,9,,2,,,,7,,,,,,,,,,
214,207,Western Sahara,9,,1,,6,,2,,15,2,,,"596,308",Africa,"66,256","596,308",
215,208,British Virgin Islands,8,,1,,7,,0,,265,33,167,"5,526","30,219",North America,"3,777","30,219",181
216,209,Papua New Guinea,8,,,,8,,0,,0.9,,"4,646",520,"8,935,646",Australia/Oceania,"1,116,956",,"1,923"


In [8]:
df_bs = df_bs.loc[8:].reset_index(drop=True)
df_bs = df_bs.drop('#', axis=1)
df_bs.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,USA,"2,026,597",+104,"113,061",+6,"773,505",+25,"1,140,031","16,907","6,125",342,"21,727,338","65,664","330,885,824",North America,163,"2,927",15
1,Brazil,"710,887",,"37,312",,"325,602",,"347,973","8,318","3,346",176,"999,836","4,706","212,467,494",South America,299,"5,694",213
2,Russia,"476,658",,"5,971",,"230,688",,"239,999","2,300","3,266",41,"13,016,023","89,193","145,930,871",Europe,306,"24,440",11
3,Spain,"288,797",,"27,136",,N/A,N/A,N/A,617,"6,177",580,"4,465,338","95,508","46,753,739",Europe,162,"1,723",10
4,UK,"287,399",,"40,597",,N/A,N/A,N/A,516,"4,235",598,"5,731,576","84,456","67,864,660",Europe,236,"1,672",12


In [9]:
df_bs = df_bs.iloc[:, :-3]

In [10]:
df_bs.columns

Index(['Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/\n1M pop\n',
       'Population', 'Continent'],
      dtype='object')

In [11]:
df_bs.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']

df_bs = df_bs[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]

df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,USA,North America,"330,885,824","2,026,597",+104,"113,061",+6,"773,505",+25,"1,140,031","16,907","6,125",342,"21,727,338","65,664"
1,Brazil,South America,"212,467,494","710,887",,"37,312",,"325,602",,"347,973","8,318","3,346",176,"999,836","4,706"
2,Russia,Europe,"145,930,871","476,658",,"5,971",,"230,688",,"239,999","2,300","3,266",41,"13,016,023","89,193"
3,Spain,Europe,"46,753,739","288,797",,"27,136",,N/A,N/A,N/A,617,"6,177",580,"4,465,338","95,508"
4,UK,Europe,"67,864,660","287,399",,"40,597",,N/A,N/A,N/A,516,"4,235",598,"5,731,576","84,456"


In [12]:
df_bs.columns[2:]

Index(['Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop'],
      dtype='object')

In [13]:
df_bs['WHO Region'] = df_bs['Country/Region'].map(who_region)
df_bs[df_bs['WHO Region'].isna()]['Country/Region'].unique()

array(['Mayotte', 'CAR', 'French Guiana', 'Diamond Princess',
       'Channel Islands', 'Réunion', 'Isle of Man', 'Martinique',
       'Faeroe Islands', 'Gibraltar', 'Cayman Islands', 'Guadeloupe',
       'Brunei ', 'Aruba', 'Sint Maarten', 'French Polynesia', 'Macao',
       'Saint Martin', 'St. Vincent Grenadines', 'Curaçao',
       'New Caledonia', 'Falkland Islands', 'Turks and Caicos',
       'Montserrat', 'MS Zaandam', 'British Virgin Islands'], dtype=object)

In [14]:
for col in df_bs.columns[2:]:
    df_bs[col] = df_bs[col].str.replace('[,+ ]', '', regex=True)
    df_bs[col] = df_bs[col].str.replace('N/A', '', regex=False)
 

df_bs.replace('', np.nan)

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,330885824,2026597,104,113061,6,773505,25,1140031,16907,6125,342,21727338,65664,Americas
1,Brazil,South America,212467494,710887,NaN,37312,NaN,325602,NaN,347973,8318,3346,176,999836,4706,Americas
2,Russia,Europe,145930871,476658,NaN,5971,NaN,230688,NaN,239999,2300,3266,41,13016023,89193,Europe
3,Spain,Europe,46753739,288797,NaN,27136,NaN,NaN,NaN,NaN,617,6177,580,4465338,95508,Europe
4,UK,Europe,67864660,287399,NaN,40597,NaN,NaN,NaN,NaN,516,4235,598,5731576,84456,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,Seychelles,Africa,98310,11,NaN,NaN,NaN,11,NaN,0,NaN,112,NaN,NaN,NaN,Africa
205,MS Zaandam,NaN,NaN,9,NaN,2,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN
206,Western Sahara,Africa,596308,9,NaN,1,NaN,6,NaN,2,NaN,15,2,NaN,NaN,Africa
207,British Virgin Islands,North America,30219,8,NaN,1,NaN,7,NaN,0,NaN,265,33,167,5526,NaN


In [15]:
df_bs.to_csv('worldometer_data.csv', index=False)